In [1]:
import sys
cpath = !pwd
sys.path.append(cpath[0][:-18])
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\')
sys.path.append('C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\')

In [2]:
from lionforests import LionForests
from algorithms.simpleSurrogate import GlobalSurrogateTree, LocalSurrogateTree
from algorithms.DefragTrees.defragTrees import DefragModel
from CHIRPS.structures import data_container
import CHIRPS.routines as rt
import CHIRPS.structures as strcts

from scipy import sparse

from datasets.dataset import Dataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd 
import numpy as np
np.seterr(invalid='ignore')
import warnings
warnings.filterwarnings("ignore")
import time

from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.metrics import f1_score, precision_score

from anchor import anchor_tabular

abalone = Dataset()

In [3]:
X, y, feature_names, class_names = abalone.load_abalone(type='classification',ys='reduced')

In [7]:
y = [int(i) for i in y]
y = np.array(y)
one_h_percent = int(min(10*len(X)/100,100))
print("Instances:",one_h_percent)

Instances: 100


In [12]:
def measure(X_train, X_test, y_train, y_test, feature_names, class_names):
    parameters = [{
        'max_depth': [10],
        'max_features': [0.75],
        'bootstrap': [True],
        'min_samples_leaf' : [10],
        'n_estimators': [10]
    }]
    lf = LionForests(None, False, None, feature_names, class_names)
    lf.fit(X_train, y_train, params=parameters)    
    train = lf.utilizer.transform(X_train)
    test = lf.utilizer.transform(X_test)

    predictions = lf.model.predict(train)
    test_predictions = lf.model.predict(test)
    
    def techniques(model, train, y_train, predictions, test, feature_names, class_names, lf, task):

        #BaselineTechnique ==============================================================================
        gt = GlobalSurrogateTree(train, predictions, feature_names, task)
        lt = LocalSurrogateTree(train, predictions, feature_names, task, 150)

        #DefragTechnique ================================================================================
        Kmax = 10
        splitter = DefragModel.parseSLtrees(model) # parse sklearn tree ensembles into the array of (feature index, threshold)
        mdl = DefragModel(modeltype=task, maxitr=100, qitr=0, tol=1e-6, restart=20, verbose=0)
        mdl.fit(train, predictions, splitter, Kmax, fittype='FAB', featurename=feature_names)
        def def_cov(instances):
            score, cover, coll = mdl.evaluate(instances[1:],lf.model.predict(instances[1:]))
            def_predictions = mdl.predict(instances[1:])
            rule = mdl.find_rule_length(instances[0])
            return rule, cover, def_predictions

        #Anchors =======================================================================================
        explainer = anchor_tabular.AnchorTabularExplainer(class_names, feature_names, train)
        def anchors_method(instance):
            exp = explainer.explain_instance(instance, lf.model.predict, threshold=0.95)
            anchors_dict = {}
            for i in exp.names():
                terms = i.split(' ')
                if len(terms) == 3:
                    anchors_dict[terms[0]] = [[terms[1],float(terms[2])]]
                else:
                    anchors_dict[terms[2]] = [[terms[3],float(terms[4])],[terms[1],float(terms[0])]]
            return anchors_dict, 0

        #CHIRPS =======================================================================================
        project_dir = 'C:\\Users\\iamollas\\Downloads\\LionForests Journal\\algorithms\\CHIRPS'
        temp_y = np.array([int(i+1) for i in y_train])
        temp_frame = pd.DataFrame(np.hstack((train,temp_y.reshape(len(temp_y),1))),columns=feature_names+['class'])
        temp_frame['class']=temp_frame['class'].astype(int)
        mydata = data_container(
                data = temp_frame, class_col = 'class', var_names = feature_names,
                project_dir = project_dir, save_dir = 'abalon', random_state=123)
        meta_data = mydata.get_meta()
        f_walker = strcts.classification_trees_walker(forest=model, meta_data=meta_data)
        f_walker.forest_walk(instances = test, labels = model.predict(test), forest_walk_async = True)

        explanations = strcts.CHIRPS_container(f_walker.path_detail,
                                        forest=model,
                                        sample_instances=sparse.csr_matrix(train), # any representative sample can be used
                                        sample_labels=predictions,
                                        meta_data=meta_data)

        explanations.run_explanations(target_classes=model.predict(test), # we're explaining the prediction, not the true label!
                                explanation_async=False,
                                random_state=123,
                                which_trees='majority',
                                alpha_paths=0.0,
                                support_paths=0.1,
                                score_func=1,
                                precis_threshold=0.99,
                                disc_path_bins=4,
                                merging_bootstraps=20,
                                pruning_bootstraps=20,
                                delta=0.2,
                                weighting='kldiv')
        def chirps_method(idx):
            chirps_dict = {}
            for i in explanations.explainers[idx].pruned_rule:
                if i[1]:
                    chirps_dict[i[0]] = [['<=',float(i[2])]]
                else:
                    chirps_dict[i[0]] = [['>',float(i[2])]]
            return chirps_dict, 0

        #LionForests
        def lf_rule(instance):
            temp = lf.train(instance)[5]
            rule = {}
            for key,value in temp.items():
                rule[key] = [['<=',value[1]],['>',value[0]]]
            return rule

        return {'gs':gt.rule,'ls':lt.rule,'an':anchors_method,'lf':lf_rule, 'df': def_cov, 'ch':chirps_method}
    interpretation = techniques(lf.model, train, y_train, predictions, test, feature_names, class_names, lf, 'classification')
    def rule_cov(instance, feature_names, rule):
        covered = True
        for k in range(len(instance)):
            feature = feature_names[k]
            if feature in rule.keys():
                if len(rule[feature]) == 2:
                    if instance[k] > rule[feature][0][1]: #<=
                        covered = False
                    if instance[k] <= rule[feature][1][1]:
                        covered = False
                elif rule[feature][0][0] == '>':
                    if instance[k] <= rule[feature][0][1]:
                        covered = False
                else:
                     if instance[k] > rule[feature][0][1]: #<=
                        covered = False
        if covered:
            return 1
        else:
            return 0

    gt = GlobalSurrogateTree(train,lf.model.predict(train),feature_names,'c')
    lt = LocalSurrogateTree(train,lf.model.predict(train),feature_names,'c',150)

    loo = LeaveOneOut()
    loo.get_n_splits(test)
    rule_generator = interpretation
    full_coverage = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    rule_length = {'gs':0, 'ls':0, 'an':0, 'lf':0, 'df':0, 'ch': 0}
    f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

    for train_index, test_index in loo.split(test):
        x_train_temp, x_test_temp = test[train_index], test[test_index]
        y_train_temp, y_test_temp = test_predictions[train_index], test_predictions[test_index]
        x_train_temp_lf = lf.utilizer.inverse_transform(x_train_temp)
        x_test_temp_lf = lf.utilizer.inverse_transform(x_test_temp)
        for name, method in rule_generator.items():
            if name == 'ch':
                rule, op = method(test_index[0])
                coverage = 0
                precision = []
                co = 0
                for i in x_train_temp_lf:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            elif name == 'df':
                #FIX RULE LENGTH!
                rule, cover, predictions = method(np.concatenate((x_test_temp,x_train_temp)))
                f_precision[name].append(precision_score(predictions,y_train_temp,average='micro'))
                full_coverage[name] = full_coverage[name] + cover
            elif name == 'lf':
                rule = method(x_test_temp_lf[0])
                coverage = 0
                precision = []
                co = 0
                for i in x_train_temp_lf:
                    res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1:
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            else:
                rule, prediction = method(x_test_temp[0])
                coverage = 0
                precision = []
                co = 0 
                for i in x_train_temp:
                    if name == 'anchors':
                        res = rule_cov(i, feature_names, rule)
                    else:    
                        res = rule_cov(i, feature_names, rule)
                    coverage = coverage + res
                    if res == 1 and name=='anchors':
                        precision.append([y_test_temp[0], y_train_temp[co]])
                    elif res == 1:
                        precision.append([prediction, y_train_temp[co]])
                    co = co + 1
                if len(precision) > 1:
                    precision = np.array(precision)
                    f_precision[name].append(precision_score(precision[:,:1],precision[:,1:],average='micro'))
                full_coverage[name] = full_coverage[name] + coverage/len(x_train_temp)
            rule_length[name] = rule_length[name] + len(rule)
    return rule_generator, full_coverage, rule_length, f_precision

In [13]:
#27 instances
total_results = []
rs = [7, 77, 777, 7777, 77777, 5, 55, 555, 5555, 55555]
for random_s in rs:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=one_h_percent, stratify=y, random_state=random_s)
    results = measure(X_train, X_test, y_train, y_test, feature_names, class_names)
    total_results.append(results)

len self.path_detail
100
Working on CHIRPS for instance 0 of 100
as_chirps for batch_idx 0
start mining for batch_idx 0 with support = 0.1
found 83 patterns from 60 trees for batch_idx 0
start score sort for batch_idx 0 (83) patterns
start merge rule for batch_idx 0 (83) patterns
[('Shell', False, 0.42937), ('Shucked', True, 0.40215)]
0.8202764976958525 0.053522928054437396 0.22691363991659205 0.31113801452784506
merge complete for batch_idx 0 (83) patterns
start get explainer for batch_idx 0
as_chirps for batch_idx 1
start mining for batch_idx 1 with support = 0.1
reduced 20 patterns out of 142 by numeric redundancy
found 122 patterns from 98 trees for batch_idx 1
start score sort for batch_idx 1 (122) patterns
start merge rule for batch_idx 1 (122) patterns
[('Shell', True, 0.25267)]
0.8953405017921147 0.31049569643456415 0.08619127837120827 0.18839427662957067
merge complete for batch_idx 1 (122) patterns
start get explainer for batch_idx 1
as_chirps for batch_idx 2
start mining for

[('Shell', True, 0.42521), ('Shell', False, 0.25755), ('Shucked', False, 0.38556)]
0.9858757062146892 0.08893655537768244 0.05526155383503134 0.07065693430656932
merge complete for batch_idx 16 (210) patterns
start get explainer for batch_idx 16
as_chirps for batch_idx 17
start mining for batch_idx 17 with support = 0.1
reduced 30 patterns out of 167 by numeric redundancy
found 137 patterns from 89 trees for batch_idx 17
start score sort for batch_idx 17 (137) patterns
start merge rule for batch_idx 17 (137) patterns
[('Shell', True, 0.18329), ('Shell', False, 0.0731)]
0.9939668174962293 0.16759918616480163 0.09825868864213415 0.17451772095109877
merge complete for batch_idx 17 (137) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18 with support = 0.1
reduced 24 patterns out of 179 by numeric redundancy
found 155 patterns from 67 trees for batch_idx 18
start score sort for batch_idx 18 (155) patterns
start merge rule for batch_idx 18

[('Shell', True, 0.24324), ('Viscera', False, 0.04651)]
0.9863013698630136 0.2929727854140544 0.07870151964460464 0.13019529293940918
merge complete for batch_idx 33 (129) patterns
start get explainer for batch_idx 33
as_chirps for batch_idx 34
start mining for batch_idx 34 with support = 0.1
reduced 3 patterns out of 260 by numeric redundancy
found 257 patterns from 100 trees for batch_idx 34
start score sort for batch_idx 34 (257) patterns
start merge rule for batch_idx 34 (257) patterns
[('Shell', True, 0.32882)]
0.8732095490716181 0.38155228023689547 0.11416939288050314 0.17576564580559245
merge complete for batch_idx 34 (257) patterns
start get explainer for batch_idx 34
Working on CHIRPS for instance 35 of 100
as_chirps for batch_idx 35
start mining for batch_idx 35 with support = 0.1
reduced 1 patterns out of 35 by numeric redundancy
found 34 patterns from 62 trees for batch_idx 35
start score sort for batch_idx 35 (34) patterns
start merge rule for batch_idx 35 (34) patterns
[(

[('Shell', False, 0.58973)]
0.8329853862212944 0.11766590479713572 0.1568159507772665 0.2238667900092509
merge complete for batch_idx 50 (72) patterns
start get explainer for batch_idx 50
as_chirps for batch_idx 51
start mining for batch_idx 51 with support = 0.1
reduced 12 patterns out of 195 by numeric redundancy
found 183 patterns from 80 trees for batch_idx 51
start score sort for batch_idx 51 (183) patterns
start merge rule for batch_idx 51 (183) patterns
[('Shell', True, 0.30478)]
0.8846815834767642 0.3672238563766634 0.13758322767698566 0.1328751431844214
merge complete for batch_idx 51 (183) patterns
start get explainer for batch_idx 51
as_chirps for batch_idx 52
start mining for batch_idx 52 with support = 0.1
reduced 6 patterns out of 84 by numeric redundancy
found 78 patterns from 62 trees for batch_idx 52
start score sort for batch_idx 52 (78) patterns
start merge rule for batch_idx 52 (78) patterns
[('Shell', False, 0.60682)]
0.8574821852731591 0.10402317872736035 0.076427

[('Shell', True, 0.13842), ('Viscera', False, 0.04666)]
0.9824561403508771 0.1288931648533248 0.12432128255373309 0.22033462033462037
merge complete for batch_idx 67 (178) patterns
start get explainer for batch_idx 67
as_chirps for batch_idx 68
start mining for batch_idx 68 with support = 0.1
reduced 22 patterns out of 199 by numeric redundancy
found 177 patterns from 84 trees for batch_idx 68
start score sort for batch_idx 68 (177) patterns
start merge rule for batch_idx 68 (177) patterns
[('Shell', True, 0.45328)]
0.8348556814029959 0.42685800731295065 0.06809106922456289 0.11908177905308465
merge complete for batch_idx 68 (177) patterns
start get explainer for batch_idx 68
as_chirps for batch_idx 69
start mining for batch_idx 69 with support = 0.1
found 26 patterns from 88 trees for batch_idx 69
start score sort for batch_idx 69 (26) patterns
start merge rule for batch_idx 69 (26) patterns
[('Viscera', True, 0.03968), ('Shell', True, 0.0326)]
0.9207920792079208 0.02464336375917623 0

...

[('Shell', True, 0.1344), ('Viscera', False, 0.05089)]
0.9774590163934426 0.12234784772418936 0.14756558336220282 0.26527657109759617
merge complete for batch_idx 17 (157) patterns
start get explainer for batch_idx 17
as_chirps for batch_idx 18
start mining for batch_idx 18 with support = 0.1
reduced 53 patterns out of 209 by numeric redundancy
found 156 patterns from 100 trees for batch_idx 18
start score sort for batch_idx 18 (156) patterns
start merge rule for batch_idx 18 (156) patterns
[('Shell', True, 0.13379), ('Viscera', False, 0.05132)]
0.9774127310061602 0.12209506291484186 0.14677242335353383 0.2659574468085103
merge complete for batch_idx 18 (156) patterns
start get explainer for batch_idx 18
as_chirps for batch_idx 19
start mining for batch_idx 19 with support = 0.1
reduced 5 patterns out of 249 by numeric redundancy
found 244 patterns from 73 trees for batch_idx 19
start score sort for batch_idx 19 (244) patterns
start merge rule for batch_idx 19 (244) patterns
[('Shell',

[('Shell', True, 0.47308)]
0.8284126432789163 0.421518669586954 0.09872270530231504 0.15423387096774185
merge complete for batch_idx 34 (127) patterns
start get explainer for batch_idx 34
Working on CHIRPS for instance 35 of 100
as_chirps for batch_idx 35
start mining for batch_idx 35 with support = 0.1
reduced 4 patterns out of 163 by numeric redundancy
found 159 patterns from 70 trees for batch_idx 35
start score sort for batch_idx 35 (159) patterns
start merge rule for batch_idx 35 (159) patterns
[('Shell', False, 0.29982), ('Shucked', True, 0.23873)]
0.8414634146341463 0.019776527385132425 0.257637462204573 0.23368298368298326
merge complete for batch_idx 35 (159) patterns
start get explainer for batch_idx 35
as_chirps for batch_idx 36
start mining for batch_idx 36 with support = 0.1
reduced 22 patterns out of 163 by numeric redundancy
found 141 patterns from 84 trees for batch_idx 36
start score sort for batch_idx 36 (141) patterns
start merge rule for batch_idx 36 (141) patterns


[('Shell', True, 0.17002), ('Viscera', False, 0.05136)]
0.9843304843304843 0.17644379692455406 0.15398552431864249 0.26787416587225954
merge complete for batch_idx 51 (145) patterns
start get explainer for batch_idx 51
as_chirps for batch_idx 52
start mining for batch_idx 52 with support = 0.1
reduced 18 patterns out of 101 by numeric redundancy
found 83 patterns from 57 trees for batch_idx 52
start score sort for batch_idx 52 (83) patterns
start merge rule for batch_idx 52 (83) patterns
[('Shell', False, 0.59969), ('Shucked', False, 0.52895), ('Height', False, 0.71857)]
0.8648648648648649 0.05495238157829653 0.23785239087429808 0.33841463414634154
merge complete for batch_idx 52 (83) patterns
start get explainer for batch_idx 52
as_chirps for batch_idx 53
start mining for batch_idx 53 with support = 0.1
reduced 2 patterns out of 27 by numeric redundancy
found 25 patterns from 88 trees for batch_idx 53
start score sort for batch_idx 53 (25) patterns
start merge rule for batch_idx 53 (2

[('Shell', True, 0.60047), ('Shucked', False, 0.55283), ('Whole', True, 0.69811), ('Viscera', True, 0.74398)]
0.896551724137931 0.07110199229599826 0.04181089791434911 0.07454200884396728
merge complete for batch_idx 68 (145) patterns
start get explainer for batch_idx 68
as_chirps for batch_idx 69
start mining for batch_idx 69 with support = 0.1
reduced 25 patterns out of 221 by numeric redundancy
found 196 patterns from 100 trees for batch_idx 69
start score sort for batch_idx 69 (196) patterns
start merge rule for batch_idx 69 (196) patterns
[('Shell', True, 0.21719)]
0.8756388415672913 0.26103878290488874 0.11665357950584879 0.20321185917232848
merge complete for batch_idx 69 (196) patterns
start get explainer for batch_idx 69
Working on CHIRPS for instance 70 of 100
as_chirps for batch_idx 70
start mining for batch_idx 70 with support = 0.1
reduced 8 patterns out of 337 by numeric redundancy
found 329 patterns from 52 trees for batch_idx 70
start score sort for batch_idx 70 (329) p

[('Shell', False, 0.3923), ('Shucked', True, 0.32593), ('Diam', True, 0.63844), ('Whole', True, 0.39726)]
0.8928571428571429 0.01321582855500323 0.20363810513329186 0.22941970310391352
merge complete for batch_idx 85 (117) patterns
start get explainer for batch_idx 85
as_chirps for batch_idx 86
start mining for batch_idx 86 with support = 0.1
reduced 11 patterns out of 71 by numeric redundancy
found 60 patterns from 59 trees for batch_idx 86
start score sort for batch_idx 86 (60) patterns
start merge rule for batch_idx 86 (60) patterns
[('Shell', False, 0.66248), ('Shucked', False, 0.55663)]
0.9223300970873787 0.05116373106496258 0.2777629086696646 0.4447476125511597
merge complete for batch_idx 86 (60) patterns
start get explainer for batch_idx 86
as_chirps for batch_idx 87
start mining for batch_idx 87 with support = 0.1
reduced 29 patterns out of 167 by numeric redundancy
found 138 patterns from 96 trees for batch_idx 87
start score sort for batch_idx 87 (138) patterns
start merge r

In [10]:
#Me 10 estims
full_coverage = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
rule_length = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch': []}
f_precision = {'gs':[], 'ls':[], 'an':[], 'lf':[], 'df':[], 'ch':[]}

for i in total_results:
    for name, method in i[0].items():
        full_coverage[name].append(i[1][name]/one_h_percent)
        rule_length[name].append(i[2][name]/one_h_percent)
        f_precision[name].append(np.array(i[3][name]).mean())
for name, method in i[0].items():
    print(name,np.array(full_coverage[name]).mean(),',',np.array(full_coverage[name]).std(),
          np.array(rule_length[name]).mean(),',',np.array(rule_length[name]).std(),
          np.array(f_precision[name]).mean(),',',np.array(f_precision[name]).std())

gs 0.062262626262626276 , 0.014135294491166612 4.197000000000001 , 0.22724656213021133 0.9859794208195783 , 0.016948544307885834
ls 0.2624545454545456 , 0.028557863652061873 2.285 , 0.1054751155486449 0.8020837671821216 , 0.017680244749459225
an 0.126959595959596 , 0.017893816088776552 3.193 , 0.25845889421724305 0.037704966174510024 , 0.016419184247965093
lf 0.00024242424242424247 , 0.0001979385650733882 6.949 , 0.09761659694949423 nan , nan
df 0.9969999999999999 , 0.004582575694955895 13.549000000000001 , 1.1495777485668377 0.7630000000000001 , 0.04754997371187473
ch 0.44703030303030306 , 0.057879884348032484 1.9740000000000002 , 0.1573658158559221 0.7675585602724629 , 0.03897094479364911
